In [8]:
import pandas as pd
import json
from dotenv import load_dotenv
from io import StringIO
import boto3
import os
load_dotenv()

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

ACCESS_KEY_ID = os.getenv("ACCESS_KEY_ID")
SECRET_ACCESS_KEY = os.getenv("SECRET_ACCESS_KEY")
S3_BUCKET = "s3-rearc-quest-hs"
S3_BLS_PREFIX = "bls/pr" 
S3_POP_KEY = "datausa/population.json" 

s3 = boto3.client(
    "s3",
    aws_access_key_id=ACCESS_KEY_ID,
    aws_secret_access_key=SECRET_ACCESS_KEY,
    region_name="us-east-1"
)

# Part 3.0
Load both the csv file from Part 1 pr.data.0.Current and the json file from Part 2 as dataframes (Spark, Pyspark, Pandas, Koalas, etc).

In [9]:
# population dataset
response = s3.get_object(Bucket=S3_BUCKET, Key=f"{S3_POP_KEY}")
pop_str = response['Body'].read().decode('utf-8')
pop_data = json.loads(pop_str)['data']
pop_df = pd.DataFrame(pop_data)
pop_df.head()

,Nation ID,Nation,Year,Population
0,01000US,United States,2013,316128839.0
1,01000US,United States,2014,318857056.0
2,01000US,United States,2015,321418821.0
3,01000US,United States,2016,323127515.0
4,01000US,United States,2017,325719178.0


In [10]:
# bls time series datasets
response = s3.get_object(Bucket=S3_BUCKET, Key=f"{S3_BLS_PREFIX}/pr.data.0.Current")
file_content = response['Body'].read().decode('utf-8')
bls_df = pd.read_csv(StringIO(file_content), sep='\t')
bls_df.head()

,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.6,NaN
1,PRS30006011,1995,Q02,2.1,NaN
2,PRS30006011,1995,Q03,0.9,NaN
3,PRS30006011,1995,Q04,0.1,NaN
4,PRS30006011,1995,Q05,1.4,NaN


# Part 3.1
Using the dataframe from the population data API (Part 2), generate the mean and the standard deviation of the annual US population across the years [2013, 2018] inclusive.

In [11]:
mean = pop_df['Population'].mean()
std = pop_df['Population'].std()

print(f"Mean of US population: {mean:.2f}")
print(f"Standard deviation of US population: {std:.2f}")

Mean of US population: 326075457.40
Standard deviation of US population: 6248735.87


# Part 3.2
Using the dataframe from the time-series (Part 1), For every series_id, find the best year: the year with the max/largest sum of "value" for all quarters in that year. Generate a report with each series id, the best year for that series, and the summed value for that year. For example, if the table had the following values:

In [12]:
bls_df.columns = [c.strip() for c in bls_df.columns]
bls_df['series_id'] = bls_df['series_id'].str.rstrip()

In [13]:
yearly_sums = bls_df.groupby(["series_id", "year"], as_index=False)["value"].sum()
result = yearly_sums.loc[yearly_sums.groupby("series_id")["value"].idxmax()]

display(result)

,series_id,year,value
27,PRS30006011,2022,20.500
58,PRS30006012,2022,17.100
65,PRS30006013,1998,705.895
108,PRS30006021,2010,17.700
139,PRS30006022,2010,12.400
174,PRS30006023,2014,503.216
213,PRS30006031,2022,20.500
243,PRS30006032,2021,17.100
251,PRS30006033,1998,702.672
306,PRS30006061,2022,37.000


# Part 3.3
Using both dataframes from Part 1 and Part 2, generate a report that will provide the value for series_id = PRS30006032 and period = Q01 and the population for that given year (if available in the population dataset). The below table shows an example of one row that might appear in the resulting table:

In [14]:
bls_filtered_df = bls_df[(bls_df['series_id'] == "PRS30006032") & (bls_df['period'] == 'Q01')]
merged_df_inner = pd.merge(
    bls_filtered_df, 
    pop_df.rename(columns={'Year': 'year'}, inplace=False)[["year", "Population"]],
    on="year",
    how='left') 
display(merged_df_inner)

,series_id,year,period,value,footnote_codes,Population
0,PRS30006032,1995,Q01,0.0,NaN,NaN
1,PRS30006032,1996,Q01,-4.2,NaN,NaN
2,PRS30006032,1997,Q01,2.8,NaN,NaN
3,PRS30006032,1998,Q01,0.9,NaN,NaN
4,PRS30006032,1999,Q01,-4.1,NaN,NaN
5,PRS30006032,2000,Q01,0.5,NaN,NaN
6,PRS30006032,2001,Q01,-6.3,NaN,NaN
7,PRS30006032,2002,Q01,-6.6,NaN,NaN
8,PRS30006032,2003,Q01,-5.7,NaN,NaN
9,PRS30006032,2004,Q01,2.0,NaN,NaN
